In [1]:
versioninfo()

Julia Version 1.5.0
Commit 96786e22cc (2020-08-01 23:44 UTC)
Platform Info:
  OS: Linux (x86_64-pc-linux-gnu)
  CPU: Intel(R) Core(TM) i7-9750H CPU @ 2.60GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-9.0.1 (ORCJIT, skylake)


ref: 

+ https://docs.julialang.org/en/v1/manual/arrays/#Broadcasting-1
+ https://docs.julialang.org/en/v1/manual/interfaces/#man-interfaces-broadcasting-1
+ https://docs.julialang.org/en/v1/base/arrays/#Broadcast-and-vectorization-1

In [2]:
mutable struct Vec2{T} <: AbstractVector{T}
    x::T
    y::T
end
Base.length(::Vec2) = 2
Base.size(::Vec2) = (2,)

Base.getindex(a::Vec2, i::Integer) = getfield(a, fieldnames(Vec2)[i])
Base.setindex!(a::Vec2{T}, v::T, i::Integer) where {T} = setfield!(a, fieldnames(Vec2)[i], v)
Base.setindex!(a::Vec2{T}, v, i::Integer) where {T} = setindex!(a, convert(T, v), i)

In [3]:
vec2 = Vec2(1.0, 2.0)

2-element Vec2{Float64}:
 1.0
 2.0

In [4]:
vec2 .+ vec2

2-element Array{Float64,1}:
 2.0
 4.0

In [5]:
struct Vec2Style <: Broadcast.AbstractArrayStyle{1} end

In [6]:
Broadcast.BroadcastStyle(::Type{<:Vec2}) = Vec2Style()

In [7]:
Vec2Style(::Val{0}) = Vec2Style()
Vec2Style(::Val{1}) = Vec2Style()
Vec2Style(::Val{M}) where {M} = Broadcast.DefaultArrayStyle{M}()

Vec2Style

In [8]:
Base.zero(::Type{Vec2{T}}) where {T} = Vec2(zero(T), zero(T))
Base.similar(bc::Broadcast.Broadcasted{Vec2Style}, ::Type{T}) where {T} = zero(Vec2{T})

In [9]:
vec2 .+ vec2

2-element Vec2{Float64}:
 2.0
 4.0

In [10]:
vec2 + vec2

2-element Vec2{Float64}:
 2.0
 4.0

In [11]:
2 .* vec2

2-element Vec2{Float64}:
 2.0
 4.0

In [12]:
2vec2

2-element Vec2{Float64}:
 2.0
 4.0

In [13]:
vec2 .+ [3, 4]

2-element Vec2{Float64}:
 4.0
 6.0

In [14]:
[3, 4] .+ vec2

2-element Vec2{Float64}:
 4.0
 6.0

In [15]:
[1 2;3 4] .+ vec2

2×2 Array{Float64,2}:
 2.0  3.0
 5.0  6.0

In [16]:
using BenchmarkTools
using Random

In [17]:
Base.rand(rng::Random.AbstractRNG, sp::Random.Sampler{Vec2{T}}) where {T} = Vec2(rand(rng, T), rand(rng, T))

In [18]:
vec2a = rand(Vec2{Float64});
vec2b = rand(Vec2{Float64});

In [19]:
@benchmark vec2a .+ vec2b

BenchmarkTools.Trial: 
  memory estimate:  1.13 KiB
  allocs estimate:  27
  --------------
  minimum time:     6.157 μs (0.00% GC)
  median time:      6.464 μs (0.00% GC)
  mean time:        6.868 μs (0.50% GC)
  maximum time:     351.483 μs (97.67% GC)
  --------------
  samples:          10000
  evals/sample:     5

In [20]:
arr2a = rand(Float64, 2);
arr2b = rand(Float64, 2);

In [21]:
@benchmark arr2a .+ arr2b

BenchmarkTools.Trial: 
  memory estimate:  160 bytes
  allocs estimate:  3
  --------------
  minimum time:     368.589 ns (0.00% GC)
  median time:      388.684 ns (0.00% GC)
  mean time:        422.199 ns (2.32% GC)
  maximum time:     20.045 μs (95.83% GC)
  --------------
  samples:          10000
  evals/sample:     207